In [28]:
import requests
import os 
import json
import numpy as np 
import pandas as pd
from datetime import datetime, timedelta
from pprint import pprint 
import pytz
import seaborn as sb
import schedule

from datetime import datetime
from datetime import timedelta
from datetime import date

import time
from PIL import Image
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor, as_completed

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
pd.options.mode.chained_assignment = None 
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, RFE, mutual_info_classif
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import asyncio
import aiohttp
import nest_asyncio


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [115]:
shots_2023 = pd.read_csv("C:\\Users\\kjcs2\\Desktop\\shots_2023\\shots_2023.csv")


,shotID,arenaAdjustedShotDistance,arenaAdjustedXCord,arenaAdjustedXCordABS,arenaAdjustedYCord,arenaAdjustedYCordAbs,averageRestDifference,awayEmptyNet,awayPenalty1Length,awayPenalty1TimeLeft,awaySkatersOnIce,awayTeamCode,awayTeamGoals,defendingTeamAverageTimeOnIce,defendingTeamAverageTimeOnIceOfDefencemen,defendingTeamAverageTimeOnIceOfDefencemenSinceFaceoff,defendingTeamAverageTimeOnIceOfForwards,defendingTeamAverageTimeOnIceOfForwardsSinceFaceoff,defendingTeamAverageTimeOnIceSinceFaceoff,defendingTeamDefencemenOnIce,defendingTeamForwardsOnIce,defendingTeamMaxTimeOnIce,defendingTeamMaxTimeOnIceOfDefencemen,defendingTeamMaxTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMaxTimeOnIceOfForwards,defendingTeamMaxTimeOnIceOfForwardsSinceFaceoff,defendingTeamMaxTimeOnIceSinceFaceoff,defendingTeamMinTimeOnIce,defendingTeamMinTimeOnIceOfDefencemen,defendingTeamMinTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMinTimeOnIceOfForwards,defendingTeamMinTimeOnIceOfForwardsSinceFaceoff,defendingTeamMinTimeOnIceSinceFaceoff,distanceFromLastEvent,event,game_id,goal,goalieIdForShot,goalieNameForShot,homeEmptyNet,homePenalty1Length,homePenalty1TimeLeft,homeSkatersOnIce,homeTeamCode,homeTeamGoals,homeTeamWon,id,isHomeTeam,isPlayoffGame,lastEventCategory,lastEventShotAngle,lastEventShotDistance,lastEventTeam,lastEventxCord,lastEventxCord_adjusted,lastEventyCord,lastEventyCord_adjusted,location,offWing,period,playerNumThatDidEvent,playerNumThatDidLastEvent,playerPositionThatDidEvent,season,shooterLeftRight,shooterName,shooterPlayerId,shooterTimeOnIce,shooterTimeOnIceSinceFaceoff,shootingTeamAverageTimeOnIce,shootingTeamAverageTimeOnIceOfDefencemen,shootingTeamAverageTimeOnIceOfDefencemenSinceFaceoff,shootingTeamAverageTimeOnIceOfForwards,shootingTeamAverageTimeOnIceOfForwardsSinceFaceoff,shootingTeamAverageTimeOnIceSinceFaceoff,shootingTeamDefencemenOnIce,shootingTeamForwardsOnIce,shootingTeamMaxTimeOnIce,shootingTeamMaxTimeOnIceOfDefencemen,shootingTeamMaxTimeOnIceOfDefencemenSinceFaceoff,shootingTeamMaxTimeOnIceOfForwards,shootingTeamMaxTimeOnIceOfForwardsSinceFaceoff,shootingTeamMaxTimeOnIceSinceFaceoff,shootingTeamMinTimeOnIce,shootingTeamMinTimeOnIceOfDefencemen,shootingTeamMinTimeOnIceOfDefencemenSinceFaceoff,shootingTeamMinTimeOnIceOfForwards,shootingTeamMinTimeOnIceOfForwardsSinceFaceoff,shootingTeamMinTimeOnIceSinceFaceoff,shotAngle,shotAngleAdjusted,shotAnglePlusRebound,shotAnglePlusReboundSpeed,shotAngleReboundRoyalRoad,shotDistance,shotGeneratedRebound,shotGoalieFroze,shotOnEmptyNet,shotPlayContinuedInZone,shotPlayContinuedOutsideZone,shotPlayStopped,shotRebound,shotRush,shotType,shotWasOnGoal,speedFromLastEvent,team,teamCode,time,timeDifferenceSinceChange,timeSinceFaceoff,timeSinceLastEvent,timeUntilNextEvent,xCord,xCordAdjusted,xFroze,xGoal,xPlayContinuedInZone,xPlayContinuedOutsideZone,xPlayStopped,xRebound,xShotWasOnGoal,yCord,yCordAdjusted
0,0,39.698866,59.0,59.0,-26.0,26.0,-3.4,0,0,0,5,NSH,0,30.0,32.0,26.0,28.666667,26.000000,26.0,2,3,33,33,26,31,26,26,26,31,26,26,26,26,56.035703,SHOT,20001,0,8477424.0,Juuse Saros,0,0,0,5,TBL,0,1,10,1.0,0,GIVE,0.000000,0.000000,AWAY,57,57,30,30,AWAYZONE,0,1,43,0,D,2023,R,Darren Raddysh,8478178.0,21,21,23.6,20.500000,20.500000,25.666667,24.000000,22.6,2,3,29,21,21,29,26,26,20,20,20,20,20,20,-40.914383,40.914383,0.000000,0.000000,0,39.698866,0,0,0,1,0,0,0,0,WRIST,1.0,18.678568,HOME,TBL,61,-6,26,3,9,59,59,0.249174,0.035813,0.379310,0.277143,0.022744,0.035816,0.751320,-26,-26
1,1,11.313708,81.0,81.0,8.0,8.0,-3.4,0,0,0,5,NSH,0,39.0,41.0,35.0,37.666667,35.000000,35.0,2,3,42,42,35,40,35,35,35,40,35,35,35,35,40.496913,SHOT,20001,0,8477424.0,Juuse Saros,0,0,0,5,TBL,0,1,11,1.0,0,SHOT,-40.914383,39.698866,HOME,59,59,-26,-26,AWAYZONE,1,1,21,43,C,2023,R,Brayden Point,8478010.0,29,29,32.6,29.500000,29.500000,34.666667,33.000000,31.6,2,3,38,30,30,38,35,35,29,29,29,29,29,29,45.000000,45.000000,85.914383,9.546043,1,11.313708,0,0,0,0,1,0,0,0,TIP,1.0,4.499657,HOME,TBL,70,-6,35,9,26,81,81,0.107945,0.057705,0.464358,0.

### Game ID Shift Charts data for 2023-2024

### Shift Data

In [2]:

# Apply nest_asyncio to allow nested event loops in Jupyter notebooks
nest_asyncio.apply()

# Initialize the starting and ending game IDs
start_game_id = 2023020001
end_game_id = 2023021307

# Base URL for the API
base_url = "https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId="
pxp_url = "https://api-web.nhle.com/v1/gamecenter/"
pxp_suffix = "/play-by-play"

# Create an empty list to store the game IDs and URLs
game_data = []

# Loop through the range of game IDs
for game_id in range(start_game_id, end_game_id + 1):
    # Create the full API URL
    api_url = f"{base_url}{game_id}"
    api_url2 = f"{pxp_url}{game_id}{pxp_suffix}"
    # Append the game ID and URL to the list
    game_data.append({"game_id": game_id, "shift_url": api_url, "pxp_url": api_url2})

# Convert the list to a DataFrame
game_df = pd.DataFrame(game_data)

async def fetch_shift_data(session, shift_url, game_id):
    try:
        async with session.get(shift_url) as response:
            if response.status == 200:
                json_data = await response.json()
                details = pd.DataFrame(json_data['data'])
                if not details.empty:
                    # Concatenate first and last names to create player_name
                    details['player_name'] = details['firstName'] + " " + details['lastName']
                    
                    # Handle empty or null eventDetails only for typeCode 505
                    mask_505 = details['typeCode'] == 505
                    if 'typeCode' in details.columns and mask_505.any():
                        details.loc[mask_505, 'eventDetails'] = details.loc[mask_505, 'eventDetails'].fillna('unassisted')
                        details.loc[mask_505, 'eventDetails'] = details.loc[mask_505, 'eventDetails'].replace(r'^\s*$', 'unassisted', regex=True)

                        # Handle splitting of eventDetails
                        assists = details.loc[mask_505, 'eventDetails'].str.split(', ', expand=True)

                        details.loc[mask_505, 'assist_1'] = assists[0]
                        details.loc[mask_505, 'assist_2'] = assists[1] if assists.shape[1] > 1 else None

                    return details
            else:
                print(f"Request failed with status code {response.status}")
                return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

async def fetch_all_shift_data(game_df):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for _, row in game_df.iterrows():
            url = row['shift_url']
            game_id = row['game_id']
            tasks.append(fetch_shift_data(session, url, game_id))
        
        results = await asyncio.gather(*tasks)
        return results

# Run the async tasks and collect the results
all_shifts_data = asyncio.run(fetch_all_shift_data(game_df))

# Concatenate all the shift data DataFrames into a single DataFrame
all_shifts = pd.concat(all_shifts_data, ignore_index=True)

all_shifts = pd.DataFrame(all_shifts)

In [3]:
#Create a dictionary to make the shot data more readable 
shot_code_dictionary = {
    801: 'slap shot',
    802: 'snap shot',
    803: 'wrist shot',
    804: 'wrap-around',
    805: 'tip-in',
    806: 'backhanded shot',
    807: 'deflected in',
    808: 'bat shot',
    809: 'cradle/Michigan',
    810: 'poke',
    811: 'between the legs'
}

all_shifts['shot_type'] = all_shifts['detailCode'].map(shot_code_dictionary)
all_shifts.head()



,id,detailCode,duration,endTime,eventDescription,eventDetails,eventNumber,firstName,gameId,hexValue,...,shiftNumber,startTime,teamAbbrev,teamId,teamName,typeCode,player_name,assist_1,assist_2,shot_type
0,13244643,0.0,00:30,00:30,None,None,7.0,Ryan,2023020001,#FFB81C,...,1,00:00,NSH,18,Nashville Predators,517,Ryan McDonagh,NaN,NaN,NaN
1,13244644,0.0,00:51,03:02,None,None,74.0,Ryan,2023020001,#FFB81C,...,2,02:11,NSH,18,Nashville Predators,517,Ryan McDonagh,NaN,NaN,NaN
2,13244645,0.0,00:44,04:34,None,None,92.0,Ryan,2023020001,#FFB81C,...,3,03:50,NSH,18,Nashville Predators,517,Ryan McDonagh,NaN,NaN,NaN
3,13244646,0.0,00:47,06:45,None,None,222.0,Ryan,2023020001,#FFB81C,...,4,05:58,NSH,18,Nashville Predators,517,Ryan McDonagh,NaN,NaN,NaN
4,13244647,0.0,00:41,09:00,None,None,250.0,Ryan,2023020001,#FFB81C,...,5,08:19,NSH,18,Nashville Predators,517,Ryan McDonagh,NaN,NaN,NaN


In [31]:

game_plays = game_plays.rename(columns = {'periodDescriptor.number':'period_number', 'periodDescriptor.periodType':'period_type', 
                                          'periodDescriptor.maxRegulationPeriods':'max_regulation_periods', 'details.eventOwnerTeamId': 'event_team_id',
                                          ' details.losingPlayerId ':'losing_player_id', 'details.winningPlayerId':'winning_player_id','details.xCoord':'xCoord',
                                          'details.yCoord':'yCoord', 'details.zoneCode':'zone_code', 'details.reason':'reason', 'details.hittingPlayerId':'hitter',
                                          'details.hitteePlayerId':'hittee', 'details.playerId':'player_id', 'details.shotType':'shot_type', 
                                          'details.shootingPlayerId':'shooting_player', 'details.goalieInNetId': 'goalie', 'details.awaySOG': 'away_sog', 
                                          'details.homeSOG': 'home_sog', 'details.blockingPlayerId':'blocker', 'details.scoringPlayerId': 'scoring_player',
                                            'details.scoringPlayerTotal':'scoring_player_total', 'details.assist1PlayerId': 'assist_1',
                                            'details.assist1PlayerTotal':'assist1_total', 'details.assist2PlayerId':'assist_2',
                                            'details.assist2PlayerTotal': 'assist2_total','details.awayScore':'away_score', 
                                            'details.homeScore':'home_score', 'details.secondaryReason':'secondary_reason',
                                            'details.typeCode':'type_code', 'details.descKey':'desc_key', 'details.duration': 'duration',
                                             'details.committedByPlayerId': 'committed_by', 'details.drawnByPlayerId':'drawn_by',
                                               'details.servedByPlayerId':'served_by'})

game_plays.head()

,game_id,eventId,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,period_number,period_type,max_regulation_periods,event_team_id,details.losingPlayerId,winning_player_id,xCoord,yCoord,zone_code,reason,hitter,hittee,player_id,shot_type,shooting_player,goalie,away_sog,home_sog,blocker,scoring_player,scoring_player_total,assist_1,assist1_total,assist_2,assist2_total,away_score,home_score,secondary_reason,type_code,desc_key,duration,committed_by,drawn_by,served_by,situation,goalie_situation
0,2023020001,102,00:00,20:00,1551,left,520,period-start,8,1,REG,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 on 5,in net
1,2023020001,101,00:00,20:00,1551,left,502,faceoff,9,1,REG,3,18.0,8478519.0,8475158.0,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 on 5,in net
2,2023020001,8,00:35,19:25,1551,left,516,stoppage,15,1,REG,3,NaN,NaN,NaN,NaN,NaN,NaN,icing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 on 5,in net
3,2023020001,103,00:35,19:25,1551,left,502,faceoff,17,1,REG,3,14.0,8476925.0,8478519.0,-69.0,22.0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 on 5,in net
4,2023020001,9,00:48,19:12,1551,left,503,hit,20,1,REG,3,18.0,NaN,NaN,64.0,42.0,D,NaN,8474568.0,8476453.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 on 5,in net


In [36]:

#Create a dictionary to make the shot data more readable 
situation_dictionary = {
    '1551': '5 on 5',
    '1451': '5 on 4',
    '1541': '5 on 4',
    '0651': '6 on 5',
    '1560': '6 on 5',
    '1441': '4 on 4',
    '1331': '3 on 3',
    '1460': '6 on 4',
    '1351': '5 on 3',     
    '0641': '6 on 4',
    '1341': '4 on 3',      
    '0101': '1 on 1',
    '1531': '5 on 3',
    '1010': '1 on 1',
    '1431': '4 on 3',
    '0440': '4 on 4',
    '0541': '5 on 4',
    '1550': '5 on 5',
    '1450': '5 on 4',
    '0551': '5 on 5',
    '0431': '4 on 3',
    '1340': '4 on 3',
    '0451': '5 on 4',
    '0531': '5 on 4',
    '0631': '6 on 3',
    '1360': '6 on 3' ,
    '1350': '5 on 4',
    '1440': '4 on 4'
}

game_plays['situation'] = game_plays['situationCode'].map(situation_dictionary)
game_plays['goalie_situation'] = np.where((game_plays['situationCode'].str.startswith('0')) | (game_plays['situationCode'].str[3] == '0'),
    'pulled', 'in net')
game_plays.head()

,game_id,eventId,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,period_number,period_type,max_regulation_periods,event_team_id,details.losingPlayerId,winning_player_id,xCoord,yCoord,zone_code,reason,hitter,hittee,player_id,shot_type,shooting_player,goalie,away_sog,home_sog,blocker,scoring_player,scoring_player_total,assist_1,assist1_total,assist_2,assist2_total,away_score,home_score,secondary_reason,type_code,desc_key,duration,committed_by,drawn_by,served_by,situation,goalie_situation
0,2023020001,102,00:00,20:00,1551,left,520,period-start,8,1,REG,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 on 5,in net
1,2023020001,101,00:00,20:00,1551,left,502,faceoff,9,1,REG,3,18.0,8478519.0,8475158.0,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 on 5,in net
2,2023020001,8,00:35,19:25,1551,left,516,stoppage,15,1,REG,3,NaN,NaN,NaN,NaN,NaN,NaN,icing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 on 5,in net
3,2023020001,103,00:35,19:25,1551,left,502,faceoff,17,1,REG,3,14.0,8476925.0,8478519.0,-69.0,22.0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 on 5,in net
4,2023020001,9,00:48,19:12,1551,left,503,hit,20,1,REG,3,18.0,NaN,NaN,64.0,42.0,D,NaN,8474568.0,8476453.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 on 5,in net


### Game Data

### Roster Data

In [68]:

#Roster Data
team_url = "https://api.nhle.com/stats/rest/en/team"

response = requests.get(team_url )
content = json.loads(response.content)

# Send an HTTP GET request to the specified URL
response = requests.get(team_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # The response content can be accessed using response.text
    response_text = response.text
    #pprint(response_text)
else:
    print(f"Request failed with status code {response.status_code}")

json_data = json.loads(response_text)
# json_data.keys()

roster = json_data['data']


In [73]:
roster

[{'id': 11,
  'franchiseId': 35,
  'fullName': 'Atlanta Thrashers',
  'leagueId': 133,
  'rawTricode': 'ATL',
  'triCode': 'ATL'},
 {'id': 34,
  'franchiseId': 26,
  'fullName': 'Hartford Whalers',
  'leagueId': 133,
  'rawTricode': 'HFD',
  'triCode': 'HFD'},
 {'id': 32,
  'franchiseId': 27,
  'fullName': 'Quebec Nordiques',
  'leagueId': 133,
  'rawTricode': 'QUE',
  'triCode': 'QUE'},
 {'id': 33,
  'franchiseId': 28,
  'fullName': 'Winnipeg Jets (1979)',
  'leagueId': 133,
  'rawTricode': 'WIN',
  'triCode': 'WIN'},
 {'id': 35,
  'franchiseId': 23,
  'fullName': 'Colorado Rockies',
  'leagueId': 133,
  'rawTricode': 'CLR',
  'triCode': 'CLR'},
 {'id': 36,
  'franchiseId': 3,
  'fullName': 'Ottawa Senators (1917)',
  'leagueId': 133,
  'rawTricode': 'SEN',
  'triCode': 'SEN'},
 {'id': 37,
  'franchiseId': 4,
  'fullName': 'Hamilton Tigers',
  'leagueId': 133,
  'rawTricode': 'HAM',
  'triCode': 'HAM'},
 {'id': 38,
  'franchiseId': 9,
  'fullName': 'Pittsburgh Pirates',
  'leagueId': 

In [103]:


df_roster= pd.DataFrame(roster)
df_roster = df_roster.convert_dtypes()
df_roster['roster_url'] = 'https://api-web.nhle.com/v1/roster/' + df_roster['triCode'] + '/20232024'
df_roster = df_roster[['id','fullName', 'triCode', 'roster_url']]
df_roster = df_roster.rename(columns = {'id':'team_id', 'fullName':'team_name', 'triCode':'tri_code'})
df_roster=df_roster.sort_values(by='team_id')


valid_team_codes = set(range(1, 11)).union(set(range(12, 27))).union(set(range(28, 31))).union(set(range(52, 53))).union(set(range(54, 56))).union(set(range(59, 60)))
filtered_rosters = df_roster[df_roster['team_id'].isin(valid_team_codes)]
filtered_rosters



,team_id,team_name,tri_code,roster_url
51,1,New Jersey Devils,NJD,https://api-web.nhle.com/v1/roster/NJD/20232024
45,2,New York Islanders,NYI,https://api-web.nhle.com/v1/roster/NYI/20232024
40,3,New York Rangers,NYR,https://api-web.nhle.com/v1/roster/NYR/20232024
27,4,Philadelphia Flyers,PHI,https://api-web.nhle.com/v1/roster/PHI/20232024
25,5,Pittsburgh Penguins,PIT,https://api-web.nhle.com/v1/roster/PIT/20232024
24,6,Boston Bruins,BOS,https://api-web.nhle.com/v1/roster/BOS/20232024
48,7,Buffalo Sabres,BUF,https://api-web.nhle.com/v1/roster/BUF/20232024
30,8,Montréal Canadiens,MTL,https://api-web.nhle.com/v1/roster/MTL/20232024
49,9,Ottawa Senators,OTT,https://api-web.nhle.com/v1/roster/OTT/20232024
50,10,Toronto Maple Leafs,TOR,https://api-web.nhle.com/v1/roster/TOR/20232024


In [104]:

# Create an empty dictionary to store the results
roster_dict = {}
roster_link = filtered_rosters['roster_url'] 

    # Define your extraction script as a function
def extract_roster_data(roster_link):
    try:
        # Send an HTTP GET request to the game URL
        response = requests.get(roster_link)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            response_text = response.text
            json_data = json.loads(response_text)

            forwards = json_data['forwards']
            forwards_df = pd.DataFrame(forwards)
            forwards_df = forwards_df[['id', 'headshot', 'firstName', 'lastName', 'sweaterNumber', 'positionCode']]
            forwards_df['firstName'] = forwards_df['firstName'].apply(lambda x: x['default'] if isinstance(x, dict) else x)
            forwards_df['lastName'] = forwards_df['lastName'].apply(lambda x: x['default'] if isinstance(x, dict) else x)
            forwards_df['player_name'] = forwards_df['firstName'] + " " + forwards_df['lastName']
            forwards_roster = forwards_df[['id', 'headshot', 'player_name', 'sweaterNumber', 'positionCode']]

            defense = json_data['defensemen']
            defense_df = pd.DataFrame(defense)
            defense_df = defense_df[['id', 'headshot', 'firstName', 'lastName', 'sweaterNumber', 'positionCode']]
            defense_df['firstName'] = defense_df['firstName'].apply(lambda x: x['default'] if isinstance(x, dict) else x)
            defense_df['lastName'] = defense_df['lastName'].apply(lambda x: x['default'] if isinstance(x, dict) else x)
            defense_df['player_name'] = defense_df['firstName'] + " " + defense_df['lastName']
            defense_roster = defense_df[['id', 'headshot', 'player_name', 'sweaterNumber', 'positionCode']]


            goalies = json_data['goalies']
            goalies_df = pd.DataFrame(goalies)
            goalies_df = goalies_df[['id', 'headshot', 'firstName', 'lastName', 'sweaterNumber', 'positionCode']]
            goalies_df['firstName'] = goalies_df['firstName'].apply(lambda x: x['default'] if isinstance(x, dict) else x)
            goalies_df['lastName'] = goalies_df['lastName'].apply(lambda x: x['default'] if isinstance(x, dict) else x)
            goalies_df['player_name'] = goalies_df['firstName'] + " " + goalies_df['lastName']
            goalie_roster = goalies_df[['id', 'headshot', 'player_name', 'sweaterNumber', 'positionCode']]
            # Append all data to form a single team roster
            team_roster_df = pd.concat([forwards_roster, defense_roster, goalie_roster], axis=0, ignore_index=True)

            team_roster_df['id'] = team_roster_df['id'] .astype(float, errors='ignore')

                                
            def extract_tricode(link):
                # Use a regular expression to find the tricode
                match = re.search(r'/([A-Z]{3})/', link)
                if match:
                    return match.group(1)
                else:
                    # Return None if no match is found
                    return None

            team_roster_df['tri_code'] = team_roster_df['headshot'].apply(extract_tricode)
            return team_roster_df  # Return the entire processed DataFrame
        else:
            print(f"Request failed for {roster_link} with status code {response.status_code}")
            return None  # Return None to indicate failure
    
    except Exception as e:
            print(f"An error occurred: {e}")
            return None  # Return None to indicate failure
    
# Loop through the rows of the filtered_rosters df
for index, row in filtered_rosters.iterrows():
    # Extract the API link from the current row
    roster_link = row['roster_url']

    # Run your game-specific data script and get the entire processed DataFrame
    team_roster = extract_roster_data(roster_link)
    
    # Check if team_roster is not None before proceeding
    if team_roster is not None:
        # Add a 'game_id' column to the game_specific_data DataFrame
        team_roster['tri_code'] = row['tri_code']

        # Store the result in the dictionary with the game ID as the key
        roster_dict[row['tri_code']] = team_roster
    else:
        print(f"Skipping row {index} due to failed request or exception.")

all_rosters = pd.concat(roster_dict.values(), ignore_index=True)

team_rosters = filtered_rosters[['team_id', 'team_name', 'tri_code']]
team_rosters = team_rosters.merge(all_rosters,  on = "tri_code", how = "left")
team_rosters = team_rosters.rename(columns= {'id':'player_id'})

# folder_path = 'C:\\Users\\kjcs2\\GitHub\\hockey\\hockey_env'
# file_name = 'rosters.csv'
# file_path2 = os.path.join(folder_path, file_name)

team_rosters.head()


An error occurred: "None of [Index(['id', 'headshot', 'firstName', 'lastName', 'sweaterNumber',\n       'positionCode'],\n      dtype='object')] are in the [columns]"
Skipping row 56 due to failed request or exception.


,team_id,team_name,tri_code,player_id,headshot,player_name,sweaterNumber,positionCode
0,1,New Jersey Devils,NJD,8479414.0,https://assets.nhle.com/mugs/nhl/20232024/NJD/...,Nathan Bastian,14.0,R
1,1,New Jersey Devils,NJD,8479407.0,https://assets.nhle.com/mugs/nhl/20232024/NJD/...,Jesper Bratt,63.0,L
2,1,New Jersey Devils,NJD,8481032.0,https://assets.nhle.com/mugs/nhl/20232024/NJD/...,Paul Cotter,NaN,C
3,1,New Jersey Devils,NJD,8481518.0,https://assets.nhle.com/mugs/nhl/20232024/NJD/...,Nolan Foote,29.0,L
4,1,New Jersey Devils,NJD,8475287.0,https://assets.nhle.com/mugs/nhl/20232024/NJD/...,Erik Haula,56.0,L


In [105]:
team_rosters.query('player_name == "David Pastrnak"')

,team_id,team_name,tri_code,player_id,headshot,player_name,sweaterNumber,positionCode
128,6,Boston Bruins,BOS,8477956.0,https://assets.nhle.com/mugs/nhl/20232024/BOS/...,David Pastrnak,88.0,R


### Game Result Data

### Shots Location Data

In [106]:
#Evaluation of shots taken in the game
shots_df = game_plays[game_plays['typeCode'].isin([505, 506, 507, 508])]
# shots_df = shots_df.sort_values(by=['timeInPeriod', 'sortOrder'])

# shots_df=shots_df.rename(columns = {'eventOwnerTeamId':'team_id'})
# shots_df = shots_df.dropna(subset=['team_id'])
# shots_df['team_id'] =shots_df['team_id'].astype('Int64').astype(int)
shots_df=shots_df[['game_id', 'eventId','timeInPeriod','situationCode', 'typeCode', 'typeDescKey', 'sortOrder', 'period_number',
'xCoord','yCoord', 'reason', 'shot_type', 'shooting_player', 'scoring_player', 'assist_1', 'assist_2', 'type_code', 'situation']]
shots_df.head()

,game_id,eventId,timeInPeriod,situationCode,typeCode,typeDescKey,sortOrder,period_number,xCoord,yCoord,reason,shot_type,shooting_player,scoring_player,assist_1,assist_2,type_code,situation
6,2023020001,63,01:01,1551,506,shot-on-goal,22,1,59.0,-26.0,NaN,wrist,8478178.0,NaN,NaN,NaN,NaN,5 on 5
7,2023020001,151,01:10,1551,506,shot-on-goal,23,1,81.0,8.0,NaN,tip-in,8478010.0,NaN,NaN,NaN,NaN,5 on 5
9,2023020001,70,01:47,1551,506,shot-on-goal,31,1,55.0,30.0,NaN,snap,8479661.0,NaN,NaN,NaN,NaN,5 on 5
16,2023020001,152,02:57,1551,506,shot-on-goal,48,1,58.0,-30.0,NaN,wrist,8479591.0,NaN,NaN,NaN,NaN,5 on 5
18,2023020001,95,03:51,1551,507,missed-shot,60,1,-64.0,34.0,wide-left,wrist,8476887.0,NaN,NaN,NaN,NaN,5 on 5


### Cumulative Goal Counts for the Season

In [109]:
goals_scored = shots_df.query('typeCode == 505')
goals_scored = goals_scored.rename(columns={'scoring_player': 'player_id'})

player_names = team_rosters[['player_id', 'player_name']]

season_goals = goals_scored.merge(player_names, on='player_id', how = "inner")


season_goal_totals = season_goals.groupby(['player_id', 'player_name'])['sortOrder'].count()
season_goal_totals=pd.DataFrame(season_goal_totals).reset_index()
season_goal_totals = season_goal_totals.rename(columns={'sortOrder': 'goals'})
season_goal_totals = season_goal_totals.sort_values(by='player_name')
season_goal_totals.head()

,player_id,player_name,goals
184,8477932.0,Aaron Ekblad,4
369,8480871.0,Adam Boqvist,1
425,8481726.0,Adam Edstrom,2
496,8484166.0,Adam Fantilli,12
268,8479323.0,Adam Fox,17


In [111]:
season_goals.head()

,game_id,eventId,timeInPeriod,situationCode,typeCode,typeDescKey,sortOrder,period_number,xCoord,yCoord,reason,shot_type,shooting_player,player_id,assist_1,assist_2,type_code,situation,player_name
0,2023020001,154,09:48,1551,505,goal,151,1,50.0,-16.0,NaN,slap,NaN,8476453.0,8475167.0,8478010.0,NaN,5 on 5,Nikita Kucherov
1,2023020001,165,16:15,1551,505,goal,490,2,81.0,1.0,NaN,tip-in,NaN,8475158.0,8476887.0,8474151.0,NaN,5 on 5,Ryan O'Reilly
2,2023020001,170,02:25,1451,505,goal,575,3,83.0,0.0,NaN,wrist,NaN,8477426.0,8478010.0,8474564.0,NaN,5 on 4,Nicholas Paul
3,2023020001,171,03:07,1010,505,goal,590,3,71.0,7.0,NaN,wrist,NaN,8479542.0,NaN,NaN,NaN,1 on 1,Brandon Hagel
4,2023020001,830,08:48,1541,505,goal,661,3,-73.0,-23.0,NaN,wrist,NaN,8478438.0,8481577.0,8474679.0,NaN,5 on 4,Tommy Novak


In [112]:
#Location of each goal

goal_locations =season_goals.sort_values(['player_name', 'player_id', 'game_id', 'sortOrder', 'period_number'], 
                                          ascending=[True, True, True, True, True])
goal_locations['goal_no'] = goal_locations.groupby('player_id').cumcount()+1

goal_locations.head()

# def adjust_coordinates(row):
#     x = row['xCoord']
#     y = row['yCoord']
#     if x < 0:
#         adj_x = abs(x)
#         adj_y = -y
#     else:
#         adj_x = x
#         adj_y = y
#     return pd.Series({'x_adjusted': adj_x, 'y_adjusted': adj_y})

# # Apply the function to each row of the DataFrame
# goal_talley[['x_adjusted', 'y_adjusted']] = goal_talley.apply(adjust_coordinates, axis=1)
# ice_map_data = goal_talley
# ice_map_data=ice_map_data[['game_id', 'goal_no', 'player_id','player_name', 'team_name', 'period', 'x_adjusted', 'y_adjusted' ]]

# # file_path3 = 'C:\\Users\\Karoline Sears\\Documents\\GitHub\\hockey-streamlit\\data\\ice_map_data.csv'
# # ice_map_data.to_csv(file_path3,  index=False, encoding='utf-8')
# print("completed", formatted_date)

,game_id,eventId,timeInPeriod,situationCode,typeCode,typeDescKey,sortOrder,period_number,xCoord,yCoord,reason,shot_type,shooting_player,player_id,assist_1,assist_2,type_code,situation,player_name,goal_no
2654,2023020530,153,04:16,1551,505,goal,72,1,-36.0,40.0,NaN,wrist,NaN,8477932.0,8478055.0,8477493.0,NaN,5 on 5,Aaron Ekblad,1
3500,2023020704,559,00:35,1551,505,goal,299,2,68.0,-28.0,NaN,snap,NaN,8477932.0,8479314.0,8477409.0,NaN,5 on 5,Aaron Ekblad,2
4137,2023020834,687,08:27,1541,505,goal,400,2,54.0,-23.0,NaN,wrist,NaN,8477932.0,8479314.0,8478542.0,NaN,5 on 4,Aaron Ekblad,3
4375,2023020876,103,00:19,1551,505,goal,14,1,-73.0,14.0,NaN,snap,NaN,8477932.0,8477409.0,8477933.0,NaN,5 on 5,Aaron Ekblad,4
4419,2023020883,623,04:41,1551,505,goal,342,2,-96.0,-8.0,NaN,backhand,NaN,8480871.0,8476432.0,8478458.0,NaN,5 on 5,Adam Boqvist,1


### Shot distances and angles

In [121]:
print(list(shots_2023.columns))

['shotID', 'arenaAdjustedShotDistance', 'arenaAdjustedXCord', 'arenaAdjustedXCordABS', 'arenaAdjustedYCord', 'arenaAdjustedYCordAbs', 'averageRestDifference', 'awayEmptyNet', 'awayPenalty1Length', 'awayPenalty1TimeLeft', 'awaySkatersOnIce', 'awayTeamCode', 'awayTeamGoals', 'defendingTeamAverageTimeOnIce', 'defendingTeamAverageTimeOnIceOfDefencemen', 'defendingTeamAverageTimeOnIceOfDefencemenSinceFaceoff', 'defendingTeamAverageTimeOnIceOfForwards', 'defendingTeamAverageTimeOnIceOfForwardsSinceFaceoff', 'defendingTeamAverageTimeOnIceSinceFaceoff', 'defendingTeamDefencemenOnIce', 'defendingTeamForwardsOnIce', 'defendingTeamMaxTimeOnIce', 'defendingTeamMaxTimeOnIceOfDefencemen', 'defendingTeamMaxTimeOnIceOfDefencemenSinceFaceoff', 'defendingTeamMaxTimeOnIceOfForwards', 'defendingTeamMaxTimeOnIceOfForwardsSinceFaceoff', 'defendingTeamMaxTimeOnIceSinceFaceoff', 'defendingTeamMinTimeOnIce', 'defendingTeamMinTimeOnIceOfDefencemen', 'defendingTeamMinTimeOnIceOfDefencemenSinceFaceoff', 'defendin

In [124]:
shots_2023.head()

,shotID,arenaAdjustedShotDistance,arenaAdjustedXCord,arenaAdjustedXCordABS,arenaAdjustedYCord,arenaAdjustedYCordAbs,averageRestDifference,awayEmptyNet,awayPenalty1Length,awayPenalty1TimeLeft,awaySkatersOnIce,awayTeamCode,awayTeamGoals,defendingTeamAverageTimeOnIce,defendingTeamAverageTimeOnIceOfDefencemen,defendingTeamAverageTimeOnIceOfDefencemenSinceFaceoff,defendingTeamAverageTimeOnIceOfForwards,defendingTeamAverageTimeOnIceOfForwardsSinceFaceoff,defendingTeamAverageTimeOnIceSinceFaceoff,defendingTeamDefencemenOnIce,defendingTeamForwardsOnIce,defendingTeamMaxTimeOnIce,defendingTeamMaxTimeOnIceOfDefencemen,defendingTeamMaxTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMaxTimeOnIceOfForwards,defendingTeamMaxTimeOnIceOfForwardsSinceFaceoff,defendingTeamMaxTimeOnIceSinceFaceoff,defendingTeamMinTimeOnIce,defendingTeamMinTimeOnIceOfDefencemen,defendingTeamMinTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMinTimeOnIceOfForwards,defendingTeamMinTimeOnIceOfForwardsSinceFaceoff,defendingTeamMinTimeOnIceSinceFaceoff,distanceFromLastEvent,event,game_id,goal,goalieIdForShot,goalieNameForShot,homeEmptyNet,homePenalty1Length,homePenalty1TimeLeft,homeSkatersOnIce,homeTeamCode,homeTeamGoals,homeTeamWon,id,isHomeTeam,isPlayoffGame,lastEventCategory,lastEventShotAngle,lastEventShotDistance,lastEventTeam,lastEventxCord,lastEventxCord_adjusted,lastEventyCord,lastEventyCord_adjusted,location,offWing,period,playerNumThatDidEvent,playerNumThatDidLastEvent,playerPositionThatDidEvent,season,shooterLeftRight,shooterName,shooterPlayerId,shooterTimeOnIce,shooterTimeOnIceSinceFaceoff,shootingTeamAverageTimeOnIce,shootingTeamAverageTimeOnIceOfDefencemen,shootingTeamAverageTimeOnIceOfDefencemenSinceFaceoff,shootingTeamAverageTimeOnIceOfForwards,shootingTeamAverageTimeOnIceOfForwardsSinceFaceoff,shootingTeamAverageTimeOnIceSinceFaceoff,shootingTeamDefencemenOnIce,shootingTeamForwardsOnIce,shootingTeamMaxTimeOnIce,shootingTeamMaxTimeOnIceOfDefencemen,shootingTeamMaxTimeOnIceOfDefencemenSinceFaceoff,shootingTeamMaxTimeOnIceOfForwards,shootingTeamMaxTimeOnIceOfForwardsSinceFaceoff,shootingTeamMaxTimeOnIceSinceFaceoff,shootingTeamMinTimeOnIce,shootingTeamMinTimeOnIceOfDefencemen,shootingTeamMinTimeOnIceOfDefencemenSinceFaceoff,shootingTeamMinTimeOnIceOfForwards,shootingTeamMinTimeOnIceOfForwardsSinceFaceoff,shootingTeamMinTimeOnIceSinceFaceoff,shotAngle,shotAngleAdjusted,shotAnglePlusRebound,shotAnglePlusReboundSpeed,shotAngleReboundRoyalRoad,shotDistance,shotGeneratedRebound,shotGoalieFroze,shotOnEmptyNet,shotPlayContinuedInZone,shotPlayContinuedOutsideZone,shotPlayStopped,shotRebound,shotRush,shotType,shotWasOnGoal,speedFromLastEvent,team,teamCode,time,timeDifferenceSinceChange,timeSinceFaceoff,timeSinceLastEvent,timeUntilNextEvent,xCord,xCordAdjusted,xFroze,xGoal,xPlayContinuedInZone,xPlayContinuedOutsideZone,xPlayStopped,xRebound,xShotWasOnGoal,yCord,yCordAdjusted
0,0,39.698866,59.0,59.0,-26.0,26.0,-3.4,0,0,0,5,NSH,0,30.0,32.0,26.0,28.666667,26.000000,26.0,2,3,33,33,26,31,26,26,26,31,26,26,26,26,56.035703,SHOT,20001,0,8477424.0,Juuse Saros,0,0,0,5,TBL,0,1,10,1.0,0,GIVE,0.000000,0.000000,AWAY,57,57,30,30,AWAYZONE,0,1,43,0,D,2023,R,Darren Raddysh,8478178.0,21,21,23.6,20.500000,20.500000,25.666667,24.000000,22.6,2,3,29,21,21,29,26,26,20,20,20,20,20,20,-40.914383,40.914383,0.000000,0.000000,0,39.698866,0,0,0,1,0,0,0,0,WRIST,1.0,18.678568,HOME,TBL,61,-6,26,3,9,59,59,0.249174,0.035813,0.379310,0.277143,0.022744,0.035816,0.751320,-26,-26
1,1,11.313708,81.0,81.0,8.0,8.0,-3.4,0,0,0,5,NSH,0,39.0,41.0,35.0,37.666667,35.000000,35.0,2,3,42,42,35,40,35,35,35,40,35,35,35,35,40.496913,SHOT,20001,0,8477424.0,Juuse Saros,0,0,0,5,TBL,0,1,11,1.0,0,SHOT,-40.914383,39.698866,HOME,59,59,-26,-26,AWAYZONE,1,1,21,43,C,2023,R,Brayden Point,8478010.0,29,29,32.6,29.500000,29.500000,34.666667,33.000000,31.6,2,3,38,30,30,38,35,35,29,29,29,29,29,29,45.000000,45.000000,85.914383,9.546043,1,11.313708,0,0,0,0,1,0,0,0,TIP,1.0,4.499657,HOME,TBL,70,-6,35,9,26,81,81,0.107945,0.057705,0.464358,0.

In [127]:
shots_df = shots_2023[['game_id', 'shooterName', 'shooterPlayerId', 'event', 'goal', 'location', 'offWing',
                       'shotAngle', 'shotAngleAdjusted', 'shotAnglePlusRebound', 'shotAnglePlusReboundSpeed', 'shotDistance', 'shotGeneratedRebound',
                       'shotPlayStopped', 'shotRebound', 'shotRush', 'shotType', 'shotWasOnGoal', 'xCord', 'xCordAdjusted', 'yCord','yCordAdjusted']]
shots_df.head()

,game_id,shooterName,shooterPlayerId,event,goal,location,offWing,shotAngle,shotAngleAdjusted,shotAnglePlusRebound,shotAnglePlusReboundSpeed,shotDistance,shotGeneratedRebound,shotPlayStopped,shotRebound,shotRush,shotType,shotWasOnGoal,xCord,xCordAdjusted,yCord,yCordAdjusted
0,20001,Darren Raddysh,8478178.0,SHOT,0,AWAYZONE,0,-40.914383,40.914383,0.000000,0.000000,39.698866,0,0,0,0,WRIST,1.0,59,59,-26,-26
1,20001,Brayden Point,8478010.0,SHOT,0,AWAYZONE,1,45.000000,45.000000,85.914383,9.546043,11.313708,0,0,0,0,TIP,1.0,81,81,8,8
2,20001,Tanner Jeannot,8479661.0,SHOT,0,AWAYZONE,0,41.423666,41.423666,0.000000,0.000000,45.343136,0,0,0,0,SNAP,1.0,55,55,30,30
3,20001,Michael Eyssimont,8479591.0,SHOT,0,AWAYZONE,1,-44.060809,44.060809,0.000000,0.000000,43.139309,0,0,0,0,WRIST,1.0,58,58,-30,-30
4,20001,Filip Forsberg,8476887.0,MISS,0,HOMEZONE,0,-53.673174,53.673174,0.000000,0.000000,42.201896,0,0,0,0,WRIST,0.0,-64,64,34,-34
